In [1]:
# import thư viện  pandas , numpy , keras , tensorflow 
from tensorflow import keras
from keras import layers 
from keras.layers import Dense 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import tensorflow_addons as tfa 

Chuẩn bị dữ liệu 

In [2]:
num_classes = 100 
input_shape = ( 32 , 32 ,3)
(x_train , y_train) ,(x_test ,y_test) = keras.datasets.cifar100.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

c:\Users\dong2\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Định cấu Hình Siêu tham số 


In [5]:
learning_rate = 0.001 
weight_decay = 0.0001
batch_size = 256 
num_epochs = 100 
image_size = 72 
patch_size = 6 
num_patches = (image_size // patch_size) **2
projection_dim  = 64
num_heads = 4
transformer_units = [
    projection_dim * 2 ,
    projection_dim,
]
transformer_layers = 8
mlp_head_units = [2048 , 1024]

Sử dụng tăng cường dữ liệu hình ảnh cho mô hình vision transformer

In [ ]:
data_augmentation = keras.Sequential(
    [
        # Thực hiện 1 lớp bình thường hóa cho hình ảnh
        layers.Normalization(),
        # Đặt lại hình ảnh với shape [image_size , image_size]
        layers.Resizing(image_size, image_size), 
        # ật ảnh với RandomFlip
        layers.RandomFlip('honrizontal'),
        # Xoay hình ảnh 
        layers.RandomRotation(factor=0.02),
        # phóng to hình ảnh 
        layers.RandomZoom(
            height_factor=0.2 , width_factor=0.2
        ),
    ], 
    # Thiết lập name = data_augment ( tăng cường dữ liệu hình ảnh)
    name = 'data_augmentation',
)
# Biến đổi thích nghi đầu vào cho phù hợp với quy chuẩn 
data_augmentation.layers[0].adapt(x_train)

Thực hiện lớp đa đầu Mlp 

In [ ]:
# Xây dựng lớp MLP tương tự như 1 lớp kết nối đầy đủ fully connected 
# nhận đầu vào là số chiều nhúng và tỷ lệ rời bỏ 
def mlp(x , hidden_units , dropout_rate):
    # Duyêth qua danh sách chứa tập hợp số lượng đơn vị units 
    for units in hidden_units:
        # Thực hiện đưa các units vào mạng Dense cho việc nhúng sâu 
        x = layers.Dense(units , activation=tf.nn.gelu)(x) # sử dụng hàm kích hoạt gelu 
        # Thêm 1 lớp rời bỏ với tỷ lệ đặt bằng tham số dropout_rate
        x = layers.Dropout(dropout_rate)(x)
    # Trả về bộ tham số vector X sau khi áp dụng lớp rời bỏ 
    return x 

Tiến hành thực hiện khởi tạo bản vá như 1 lớp 

In [ ]:
# Xây dựng 1 lớp phương thức sử dụng cho việc trích xuất các bản vá từ hình ảnh
# ban đầu tra về các bản với với kích thước patch_size , patch_size , batch
class Patches(layers.Layer):
    # Thiết lập phương thức khởi tạo nhận tha số đầu vào là patch_size 
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size 
        # Xây dựng phương thức call thực hiện cho công việc trích xuất bản vá
    def call(self, images):
        # Lấy ra kích thước lô từ nguồn đầu vào 
        batch_size = tf.shape(images)[0]
        # sử dụng tf.image.extract_patches để tríc xuất và đặt quy chuẩn cho các  bản vá 
        patches = tf.image.extract_patches(
            # Nhận nguồn là ảnh đầu vào 
            images = images , 
            sizes = [ 1 , self.patch_size, patch_size, 1],
            strides = [1 , self.patch_size , self.patch_size, 1],
            rates = [1 , 1 , 1 , 1],
            padding = 'VALID',
        )
        # Lấy ra số chiều nhúng 
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches , [batch_size , -1 , patch_dims])
        return patches 

In [ ]:

import matplotlib.pyplot as plt 
plt.figure(figsize=(4 , 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]

plt.imshow(image.astype('uint8'))
plt.axis('off')

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]) , size =(image_size ,image_size)
)

patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")


In [ ]:
n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4 ,4))
for i , patch in enumerate(patches[0]):
    ax = plt.subplot(n , n , i+1)
    patch_img = tf.reshape(patch , (patch_size , patch_size ,3))
    plt.imshow(patch_img.numpy().astype('uint8'))
    plt.axis('off')

Xây dựng lớp mã hóa bản vá lỗi 

In [ ]:
class PatchEncoder(layers.layer):
    def __init__(self, num_patches ,projection_dim):
        self.num_patches = num_patches 
        self.proj = layers.Dense(projection_dim)
        self.pos_embed = layers.Embedding(
            input_dim=num_patches , output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start = 0 , limit=self.num_patches , delta=1)
        encode = self.proj(patch) + self.pos_embed(positions)
        return encode 

Xây dựng mô hình vision Transformer

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # augment data
    augmented = data_augmentation(inputs)
    # create patches 
    patches = Patches(patch_size)(augmented)

    # encode patches 
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # create multi layers of the transformer block 
    for _ in range (transformer_layers):
        # layers normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # create multihead attention layers 
        attention_output = layers.MultiHeadAttention(
            num_heads=num_classes, key_dim = projection_dim , dropout=0.1
        )(x1, x1)

        # add skip connection 1
        x2 = layers.Add()([attention_output , encoded_patches])
        # add layer normal 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # add mlp 
        x3 = mlp(x3 ,hidden_units=transformer_units , dropout_rate=0.1)
        # skip connection 2
        encoded_patches = layers.Add()([x3 , x2])

    #crearte a [batch_size , projection_dim] tensor 
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)

    # addd mlp 
    features = mlp(representation , hidden_units=mlp_head_units , dropout_rate=0.5)

    # classifier output 
    logits = layers.Dense(num_classes)(features)
    
    # creaet the keras.modes
    model = keras.Model(inputs=inputs , outputs=logits)
    return model 


Huấn luyện , tối ưu , và đánh giá mô hình 

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)